# 飞书消息自动翻译系统

## 功能说明
- 接收飞书消息文本
- 自动检测语言类型
- 将中文翻译为英文，英文翻译为中文
- 通过FastAPI提供HTTP接口服务
- 使用ngrok暴露公网访问地址
- 与n8n工作流集成，实现自动化处理

## 技术栈
- **transformers**: 提供预训练的翻译模型
- **FastAPI**: 构建RESTful API服务
- **uvicorn**: ASGI服务器
- **pyngrok**: 内网穿透工具
- **langdetect**: 语言检测库
- **MarianMT**: 神经机器翻译模型

## 使用说明
1. 依次运行所有代码单元
2. 配置ngrok token
3. 启动翻译API服务
4. 获取ngrok公网地址
5. 在n8n中配置该地址
6. 通过飞书机器人发送消息进行测试


In [ ]:
# 安装必要的依赖包
# transformers: 提供预训练的翻译模型
# torch: PyTorch深度学习框架
# fastapi: 现代、快速的Web框架
# uvicorn: ASGI服务器
# pyngrok: Python的ngrok包装器
# langdetect: 语言检测库
# sentencepiece: 用于某些翻译模型的分词器

%pip install transformers==4.35.0 torch fastapi uvicorn pyngrok langdetect sentencepiece protobuf requests sacremoses
print("✅ 依赖包安装完成")


In [ ]:
# 导入标准库
import json
import threading
import time
import os
from typing import Dict, Any, Optional

# 导入第三方库
from fastapi import FastAPI, HTTPException, Request
from fastapi.responses import JSONResponse
from pydantic import BaseModel
import uvicorn
from transformers import MarianMTModel, MarianTokenizer, pipeline
from langdetect import detect, DetectorFactory
from pyngrok import ngrok
import torch

# 设置语言检测的随机种子，确保结果一致性
DetectorFactory.seed = 0

print("✅ 所有库导入成功")
print(f"📱 PyTorch版本: {torch.__version__}")
print(f"🔧 设备: {'GPU' if torch.cuda.is_available() else 'CPU'}")


In [ ]:
# 配置ngrok认证token
# 请替换为您在 https://ngrok.com 注册后获得的token
NGROK_TOKEN = "your_ngrok_token_here"  # 请替换为您的实际token

# 配置ngrok认证
if NGROK_TOKEN and NGROK_TOKEN != "your_ngrok_token_here":
    import subprocess
    subprocess.run(["ngrok", "config", "add-authtoken", NGROK_TOKEN])
    print("✅ ngrok认证配置成功")
else:
    print("⚠️ 请设置您的ngrok token")
    print("🔗 请访问 https://ngrok.com 注册并获取token")
    print("📝 将token替换到上面的NGROK_TOKEN变量中")


In [ ]:
# 翻译模型类
class TranslationService:
    """翻译服务类，负责加载模型和执行翻译任务"""
    
    def __init__(self):
        """初始化翻译服务，加载中英互译模型"""
        print("🔄 正在加载翻译模型...")
        
        # 中文到英文的翻译模型
        self.zh_to_en_model_name = "Helsinki-NLP/opus-mt-zh-en"
        # 英文到中文的翻译模型  
        self.en_to_zh_model_name = "Helsinki-NLP/opus-mt-en-zh"
        
        try:
            # 加载中文到英文的模型和分词器
            print("📥 加载中文→英文翻译模型...")
            self.zh_to_en_tokenizer = MarianTokenizer.from_pretrained(self.zh_to_en_model_name)
            self.zh_to_en_model = MarianMTModel.from_pretrained(self.zh_to_en_model_name)
            
            # 加载英文到中文的模型和分词器
            print("📥 加载英文→中文翻译模型...")
            self.en_to_zh_tokenizer = MarianTokenizer.from_pretrained(self.en_to_zh_model_name)
            self.en_to_zh_model = MarianMTModel.from_pretrained(self.en_to_zh_model_name)
            
            print("✅ 翻译模型加载完成")
            
        except Exception as e:
            print(f"❌ 模型加载失败: {e}")
            raise
    
    def detect_language(self, text: str) -> str:
        """检测文本语言
        
        Args:
            text: 待检测的文本
            
        Returns:
            语言代码 ('zh-cn', 'en', 'other')
        """
        try:
            # 使用langdetect检测语言
            detected_lang = detect(text)
            
            # 映射检测结果到我们支持的语言
            if detected_lang in ['zh', 'zh-cn', 'zh-tw']:
                return 'zh-cn'
            elif detected_lang == 'en':
                return 'en'
            else:
                # 对于其他语言，我们默认当作英文处理
                return 'en'
                
        except Exception as e:
            print(f"⚠️ 语言检测失败: {e}，默认使用英文")
            return 'en'
    
    def translate_text(self, text: str, source_lang: str = None) -> Dict[str, Any]:
        """翻译文本
        
        Args:
            text: 待翻译的文本
            source_lang: 源语言(可选，不提供则自动检测)
            
        Returns:
            包含翻译结果的字典
        """
        try:
            # 如果没有指定源语言，则自动检测
            if source_lang is None:
                source_lang = self.detect_language(text)
            
            print(f"🔍 检测到语言: {source_lang}")
            print(f"📝 原文: {text}")
            
            # 根据检测到的语言选择翻译方向
            if source_lang == 'zh-cn':
                # 中文翻译为英文
                target_lang = 'en'
                inputs = self.zh_to_en_tokenizer(text, return_tensors="pt", padding=True)
                translated = self.zh_to_en_model.generate(**inputs, max_length=512)
                translated_text = self.zh_to_en_tokenizer.decode(translated[0], skip_special_tokens=True)
                
            else:
                # 英文翻译为中文
                target_lang = 'zh-cn'
                inputs = self.en_to_zh_tokenizer(text, return_tensors="pt", padding=True)
                translated = self.en_to_zh_model.generate(**inputs, max_length=512)
                translated_text = self.en_to_zh_tokenizer.decode(translated[0], skip_special_tokens=True)
            
            print(f"🎯 译文: {translated_text}")
            
            return {
                'success': True,
                'original_text': text,
                'translated_text': translated_text,
                'source_language': source_lang,
                'target_language': target_lang,
                'model_used': self.zh_to_en_model_name if source_lang == 'zh-cn' else self.en_to_zh_model_name
            }
            
        except Exception as e:
            print(f"❌ 翻译失败: {e}")
            return {
                'success': False,
                'error': str(e),
                'original_text': text
            }

# 初始化翻译服务
translation_service = TranslationService()


In [ ]:
# 定义请求和响应的数据模型
class TranslationRequest(BaseModel):
    """翻译请求模型"""
    text: str  # 待翻译的文本
    source_language: Optional[str] = None  # 源语言(可选)

class TranslationResponse(BaseModel):
    """翻译响应模型"""
    success: bool
    original_text: str
    translated_text: Optional[str] = None
    source_language: Optional[str] = None
    target_language: Optional[str] = None
    model_used: Optional[str] = None
    error: Optional[str] = None

# 创建FastAPI应用
app = FastAPI(
    title="飞书消息翻译API",
    description="自动检测语言并进行中英互译的API服务",
    version="1.0.0"
)

@app.get("/")
async def root():
    """根路径，返回API基本信息"""
    return {
        "message": "飞书消息翻译API",
        "version": "1.0.0",
        "endpoints": {
            "/translate": "POST - 翻译文本",
            "/health": "GET - 健康检查",
            "/docs": "GET - API文档"
        }
    }

@app.get("/health")
async def health_check():
    """健康检查接口"""
    return {
        "status": "healthy",
        "timestamp": time.time(),
        "service": "translation-api"
    }

@app.post("/translate", response_model=TranslationResponse)
async def translate_text(request: TranslationRequest):
    """翻译文本接口
    
    Args:
        request: 包含待翻译文本的请求
        
    Returns:
        翻译结果
    """
    try:
        # 验证输入
        if not request.text or not request.text.strip():
            raise HTTPException(status_code=400, detail="文本内容不能为空")
        
        # 执行翻译
        result = translation_service.translate_text(
            text=request.text.strip(),
            source_lang=request.source_language
        )
        
        return TranslationResponse(**result)
        
    except HTTPException:
        raise
    except Exception as e:
        print(f"❌ API错误: {e}")
        raise HTTPException(status_code=500, detail=f"翻译服务错误: {str(e)}")

@app.post("/feishu/translate")
async def feishu_translate(request: Request):
    """专为飞书设计的翻译接口
    
    接收飞书webhook消息并返回翻译结果
    """
    try:
        # 解析请求数据
        data = await request.json()
        print(f"📨 收到飞书消息: {data}")
        
        # 提取消息文本(适配飞书消息格式)
        message_text = ""
        if 'event' in data and 'message' in data['event']:
            # 飞书事件格式
            message_content = data['event']['message'].get('content', '{}')
            try:
                content_json = json.loads(message_content)
                message_text = content_json.get('text', '')
            except:
                message_text = message_content
        elif 'text' in data:
            # 简单文本格式
            message_text = data['text']
        elif 'message' in data:
            # 消息格式
            message_text = data['message']
        
        if not message_text:
            return JSONResponse({
                "success": False,
                "error": "未找到有效的消息文本"
            })
        
        # 执行翻译
        result = translation_service.translate_text(message_text)
        
        # 返回飞书友好的格式
        return JSONResponse({
            "success": result['success'],
            "original": result['original_text'],
            "translation": result.get('translated_text', ''),
            "source_lang": result.get('source_language', ''),
            "target_lang": result.get('target_language', ''),
            "reply_text": f"🔄 翻译结果\\n原文: {result['original_text']}\\n译文: {result.get('translated_text', '')}"
        })
        
    except Exception as e:
        print(f"❌ 飞书接口错误: {e}")
        return JSONResponse({
            "success": False,
            "error": str(e)
        })

print("✅ FastAPI应用创建完成")
print("📋 可用接口:")
print("  - GET  /         : API基本信息")
print("  - GET  /health   : 健康检查")
print("  - POST /translate: 通用翻译接口")
print("  - POST /feishu/translate: 飞书专用翻译接口")


In [ ]:
# 配置服务参数
HOST = "0.0.0.0"
PORT = 8000

def run_server():
    """在后台线程中运行FastAPI服务器"""
    uvicorn.run(
        app, 
        host=HOST, 
        port=PORT, 
        log_level="info",
        access_log=True
    )

# 在后台线程启动服务器
print(f"🚀 启动API服务器在 {HOST}:{PORT}...")
server_thread = threading.Thread(target=run_server, daemon=True)
server_thread.start()

# 等待服务器启动
print("⏳ 等待服务器启动...")
time.sleep(3)

# 启动ngrok隧道
try:
    print("🌐 启动ngrok隧道...")
    public_tunnel = ngrok.connect(PORT)
    public_url = public_tunnel.public_url
    
    print("\\n" + "="*60)
    print("🎉 服务启动成功!")
    print("="*60)
    print(f"📍 本地地址: http://{HOST}:{PORT}")
    print(f"🌍 公网地址: {public_url}")
    print("="*60)
    print("\\n📋 API接口地址:")
    print(f"  🏠 首页:           {public_url}/")
    print(f"  💚 健康检查:       {public_url}/health")
    print(f"  📖 API文档:        {public_url}/docs")
    print(f"  🔄 通用翻译:       {public_url}/translate")
    print(f"  📱 飞书翻译:       {public_url}/feishu/translate")
    print("\\n🔧 在n8n中使用以下URL:")
    print(f"   {public_url}/feishu/translate")
    print("="*60)
    
    # 保存URL到文件，方便后续使用
    with open('/tmp/ngrok_url.txt', 'w') as f:
        f.write(public_url)
    
    print("✅ 服务地址已保存到 /tmp/ngrok_url.txt")
    
except Exception as e:
    print(f"❌ ngrok启动失败: {e}")
    print("📝 请检查ngrok token是否配置正确")


In [ ]:
import requests
import json

# 获取服务地址
try:
    with open('/tmp/ngrok_url.txt', 'r') as f:
        base_url = f.read().strip()
except:
    base_url = f"http://localhost:{PORT}"

print(f"🔧 测试API地址: {base_url}")
print("\\n" + "="*50)
print("🧪 开始API测试")
print("="*50)

# 测试用例
test_cases = [
    {"text": "你好，世界！", "description": "中文→英文"},
    {"text": "Hello, how are you?", "description": "英文→中文"},
    {"text": "今天天气很好", "description": "中文日常用语"},
    {"text": "I love programming", "description": "英文日常用语"}
]

# 执行测试
for i, test_case in enumerate(test_cases, 1):
    print(f"\\n📝 测试 {i}: {test_case['description']}")
    print(f"原文: {test_case['text']}")
    
    try:
        # 发送翻译请求
        response = requests.post(
            f"{base_url}/translate",
            json={"text": test_case['text']},
            timeout=30
        )
        
        if response.status_code == 200:
            result = response.json()
            print(f"✅ 译文: {result['translated_text']}")
            print(f"🔍 检测语言: {result['source_language']} → {result['target_language']}")
        else:
            print(f"❌ 请求失败: {response.status_code} - {response.text}")
            
    except requests.exceptions.RequestException as e:
        print(f"❌ 网络错误: {e}")
    except Exception as e:
        print(f"❌ 测试错误: {e}")

print("\\n" + "="*50)
print("🧪 API测试完成")
print("="*50)

# 测试飞书接口格式
print("\\n📱 测试飞书接口格式:")
feishu_test_data = {
    "text": "这是一条测试消息"
}

try:
    response = requests.post(
        f"{base_url}/feishu/translate",
        json=feishu_test_data,
        timeout=30
    )
    
    if response.status_code == 200:
        result = response.json()
        print(f"✅ 飞书格式测试成功")
        print(f"📨 回复消息: {result.get('reply_text', '')}")
    else:
        print(f"❌ 飞书接口测试失败: {response.status_code}")
        
except Exception as e:
    print(f"❌ 飞书接口测试错误: {e}")


In [ ]:
print("🔄 翻译服务正在运行中...")
print("💡 提示:")
print("  - 在n8n中使用上面显示的公网地址")
print("  - 服务会持续运行直到停止此单元格")
print("  - 可以通过 /docs 查看完整API文档")
print("  - 按 Ctrl+C 或停止单元格来终止服务")
print("\\n🏃‍♂️ 按住Shift+Enter重新运行此单元格以查看状态")

# 显示当前时间和服务状态
import datetime
current_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
print(f"\\n⏰ 当前时间: {current_time}")
print(f"🌐 服务地址: {public_url if 'public_url' in locals() else '未启动'}")
print(f"💾 线程状态: {'运行中' if server_thread.is_alive() else '已停止'}")

# 获取ngrok隧道信息
try:
    tunnels = ngrok.get_tunnels()
    print(f"🔗 活跃隧道数: {len(tunnels)}")
    for tunnel in tunnels:
        print(f"  📍 {tunnel.name}: {tunnel.public_url}")
except:
    print("⚠️ 无法获取隧道信息")

print("\\n" + "="*60)
print("🎯 服务运行中，请在n8n中配置webhook地址")
print("="*60)
